In [ ]:
#######################################################################
# This script was written by Nana Owusu, it is meant to perform T1rho #
# estimation from NIfTI image files & convert to NIfTI image files.   #
#######################################################################

import os, sys, fnmatch, tkinter
import ctypes as ctyp
import numpy as np
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tkinter import filedialog
from matplotlib import rc

## Section for loading NIfTI files

In [ ]:
class INT32CAST(ctyp.Structure):
    _fields_ = [("count", ctyp.c_int),
                ("values", ctyp.POINTER(ctyp.c_int32))]

class FLOATCAST(ctyp.Structure):
    _fields_ = [("count", ctyp.c_int),
                ("values", ctyp.POINTER(ctyp.c_float))]

# Function for picking NIfTI file order
def uigetdir(tsl2Disp):
    # initialize tkinter object
    main_win = tkinter.Tk()
#     main_win.geometry("500x500")
    
    # starting from user's Documents folder 
    sourceFolder =  \
    filedialog.askdirectory(parent=main_win, initialdir= "/Users/nowusu/Documents/",\
                            title='Please select TSL {0} directory'.format(str(tsl2Disp)))

    # run the GUI program
    main_win.mainloop()
#     main_win.withdraw()
    
    return sourceFolder

def getFidAllScalar(parentDir):
    '''This function extracts the image scaling factor
    from the DICOM header. This is a scaling factor of
    signal intensities used in FIDAll. FIDAll pixel 
    values must be divided by this value to obtain 
    their true relative intensity
    (SJK implemented this in MATLAB first).'''
    
    import pydicom as pdcm

    # provide the absolute path of the current directory
    imLoc = parentDir + '/DICOM/'
    # list all files in the DICOM directory
    dirList = os.listdir(imLoc)
    # store only NIfTI files
    filesInDir = fnmatch.filter(dirList,'*.dcm')[0]
    
    dcmIm = pdcm.dcmread(imLoc + filesInDir)
    scalar = float(dcmIm[0x19,0x10b7].value)
    
    del dcmIm
    
    return scalar

def getEfgreScalar(parentDir):
    ''' This function extracts the field 'rhuser2' from
    the dicom header by reading the header, converting
    the appropriate position to float, and stripping
    everything else.
    (SJK implemented this in MATLAB first)'''
    
    import pydicom as pdcm
    import struct as st

    # provide the absolute path of the current directory
    imLoc = parentDir + '/DICOM/'
    # list all files in the DICOM directory
    dirList = os.listdir(imLoc)
    # store only NIfTI files
    filesInDir = fnmatch.filter(dirList,'*.dcm')[0]
    
    dcmIm = pdcm.dcmread(imLoc + filesInDir)
    b2Int = st.iter_unpack("@B",dcmIm[0x43,0x102a].value) # B is for unsigned character

    tpsInfo = []

    for x in b2Int:
        tpsInfo.append(x[0])

    # This is a method of doing the same typecasting with Numpy
    # rh_offset = np.array(tpsInfo[24:28],dtype=np.uint8)
    # rh_offset.view(np.int32)

    ar2Cast = INT32CAST()
    ar2Cast.count = 4
    ar2Cast.values = ctyp.cast((ctyp.c_uint8 * 4)(tpsInfo[24],tpsInfo[25],tpsInfo[26],tpsInfo[27]),
                               ctyp.POINTER(ctyp.c_int32))
    
    rh_offset = ar2Cast.values[0]

    int2Cast = FLOATCAST()
    int2Cast.count = 4
    int2Cast.values = ctyp.cast((ctyp.c_uint8 * 4)(tpsInfo[rh_offset+72*4],tpsInfo[rh_offset+72*4+1],
                                                  tpsInfo[rh_offset+72*4+2],tpsInfo[rh_offset+72*4+3]),
                               ctyp.POINTER(ctyp.c_float))

    rhuser2 = int2Cast.values[0]
    
    del ar2Cast, int2Cast, dcmIm, b2Int, tpsInfo
    
    return rhuser2

In [ ]:
## T1 correction factor for 3DFSE
def t1_correction_fse(sl_time):
    assumed_T1 = 1200
    
    for i in sl_time:
        if i == 5:
            t1r_recovery_time = 2980650 - 420154
        elif i == 80:
            t1r_recovery_time = 2980650 - 494297

        protTrec_in_us = t1r_recovery_time
        protTSL_in_us = i*1000
        protTau_in_us = 2*1068 + 3132

        tsl = float(protTSL_in_us*0.001)
        t1rho = float(75)
        trec = float(protTrec_in_us*0.001)
        T1 = float(assumed_T1)
        tau = float(protTau_in_us*0.001)

        M0 = 1
        Mz0 = M0*np.exp(-tsl/t1rho)*(1-np.exp(-trec/T1))
        tau1 = Mz0
        Mz0 = Mz0 + (M0 - Mz0)*(1-np.exp(-tau/T1))
        tau2 = Mz0

        t1_factor = tau1/tau2
        yield t1_factor
    

# Start of Image Processing

In [ ]:
# Call GUI window for selection of DICOM directories
# for storage of their full paths
seqName = 'fse'
temporalInterp = 0
TSLs = [10,50]
niiList = []
for i in range(len(TSLs)):
    niiList.append(uigetdir(TSLs[i]))

In [ ]:
# Create placeholder and fill list for image in acquisition
niiFiles = []
imScalar = []
for j in range(len(niiList)):
    # provide the absolute path of the current directory
    imLoc = niiList[j]
    # list all files in the DICOM directory
#     dirList = os.listdir(imLoc + '/aligned/')
    dirList = os.listdir(imLoc)
    # store only NIfTI files
    filesInDir = fnmatch.filter(dirList,'*.nii')
#     filesInDir = fnmatch.filter(dirList,'*align.nii.gz')
    
    if 'fid' in seqName:
        imScalar.append(getFidAllScalar(niiList[j]))
    elif 'gre' in seqName:
        imScalar.append(getEfgreScalar(niiList[j]))
    else:
        #fse_scalar = t1_correction_fse(TSLs)
        fse_scalar = [1,1]
        for i in fse_scalar:
            imScalar.append(i)
        del fse_scalar

    for i in range(len(filesInDir)):
        niiFiles.append(filesInDir[i])

imsAcquired = len(niiList)

# Create multidimentional list for storted abs. paths
# NIfTI images
niiFullPath = []
for i,j in enumerate(niiFiles):
    #fill array with nii abs. paths
#     niiFullPath.append(niiList[i] + '/aligned/'+ j)
    niiFullPath.append(niiList[i] + '/' + j)
    
# Garbage collection of unused variables
del niiList, dirList, niiFiles

## Load files

In [ ]:
niiFullPath

In [ ]:
def sliceSorter(seqName,image_data,vol,seriesFiles,scalar):
    if 'fse' in seqName:
        for i,j in enumerate(seriesFiles):
            img = nib.load(j)
            vol[:,:,:,i] = scalar[i] * img.get_data()
    elif 'fid' in seqName:
        for i,j in enumerate(seriesFiles):
            img = nib.load(j)
            vol[:,:,:,i] = scalar[i] * img.get_data()
    elif 'gre' in seqName:
        for i,j in enumerate(seriesFiles):
            img = nib.load(j)
            vol[:,:,:,i] = scalar[i] * img.get_data()
    else:
        raise UserWarning('seqName is must contain "fse","fid" or "gre"')

# load a spin-lock image for info on scan parameters
niiObj = nib.load(niiFullPath[0])

# get image data and dimension info 
imgData = niiObj.get_data()
imHdr = niiObj.header
imSize = np.shape(imgData)
imAffine = niiObj.affine

# create placeholder for spin-lock volume array
tslVol = np.zeros((imSize[0], imSize[1], imSize[2], imsAcquired), dtype='float32')

sliceSorter(seqName, imgData, tslVol, niiFullPath, imScalar)

## Save sorted spin-lock images

In [ ]:
# DICOM to NIfTI conversion info
def dcm2niConvert(affine, imToSave, saveAs):
    # Make NIfTI image object and save it
    tslIm = nib.Nifti1Image(imToSave, affine)
    nib.save(tslIm, saveAs)

## Calculate T1rho maps

### Functions for calculating T1rho and masking

In [ ]:
def maskProg(unMasked,thresh,case):
    imgDim = unMasked.shape
    m, n, numOfSlices = imgDim[1], imgDim[2], imgDim[0]
#     m, n, numOfSlices = imgDim[0], imgDim[2], imgDim[1]
    
    meanVol = np.zeros((numOfSlices, m, n))
    mask = np.zeros((numOfSlices, m, n))
#     meanVol = np.zeros((n, numOfSlices, m))
#     mask = np.zeros((n, numOfSlices, m))

    if case == 0:
        for l in range(numOfSlices):
            avgThruT = np.mean(unMasked[l,:,:,0], axis=0)
            avgVal = [mu for mu in avgThruT]
            meanVol[l,:,:] = avgVal
        brainPix = meanVol > thresh
        mask[brainPix] = 1
    elif case == 1:
        buffer = np.zeros((m,n))
        for l in range(numOfSlices):
            maskBuffer = np.squeeze(unMasked[l,:,:,0])
#             maskBuffer = np.squeeze(unMasked[:,l,:,0])
            buffer[maskBuffer > 2.5*np.median(maskBuffer.ravel('F'))] = 1
            mask[l,:,:] = buffer
#             mask[:,l,:] = buffer
        del buffer, maskBuffer
    elif case == 2:
        mask[mask == 0] = 1
    
    del m, n, numOfSlices, imgDim
    return [meanVol, mask]

def productFunc(mat1,mat2):
    prod = np.multiply(mat1,mat2)
    return prod

def applyMask(img, mask2Use):
    dims1 = np.ndim(img)
    dims2 = np.ndim(np.squeeze(mask2Use))
    matSize = len(mask2Use.flatten('F'))
    
    if dims1 > dims2:
        volProd = np.zeros((img.shape))
        
        doProduct = np.vectorize(productFunc,otypes=[np.float],cache=False)        
        for idx in range(img.shape[3]):
            volProd[:,:,:,idx] = doProduct(img[:,:,:,idx], mask2Use)
    else:
        volProd = np.multiply(img, mask2Use)
        
    return volProd

def t1rhoCalc(non0Len, indices, TSL, tslIms, t1rVolBuffer, twoTSL):
    #np.seterr(divide='warn', invalid='warn')
    if twoTSL == 1:
        sigOut = np.array(np.zeros((2,non0Len)))
        dTSL = TSL[1] - TSL[0]
        sigOut[0,:] = tslIms[:,:,0].ravel('A').take(indices)
        sigOut[1,:] = tslIms[:,:,1].ravel('A').take(indices)
        
#         print(np.shape(tslIms[:,:,0].ravel('A').take(indices)))
#         print(np.shape(tslIms[:,:,1].ravel('A').take(indices)))
        
        t1rVolBuffer.ravel(order='A')[indices] = \
            -dTSL/np.log(np.true_divide(sigOut[1,:],sigOut[0,:],order='A'),order='A')
    else:
        for ni in range(non0Len):
            for i in range(len(TSLs)):
                sigOut = tslIms(np.add(indices[ni], indexOne[0:tslPlayed]))
            P = np.polyfit(TSLs, np.log(sigOut), 1)
            t1rVolBuffer.ravel(order='A')[indices[ni]] = -1/P[0]
    return t1rVolBuffer

### Calculation

In [ ]:
matrix_size = np.shape(tslVol)
tslPlayed = len(TSLs)

mean , t1rMask = maskProg(tslVol,1250,2)
# t1rMask = nib.load("/Volumes/mrrcdata/Reliability_NKO/StudyData/20190318/subjDir_fse/20190318/S")
maskedTslBuffer = applyMask(tslVol,t1rMask)
del mean

In [ ]:
Nx, Ny, Ns = imSize[1], imSize[2], imSize[0]
# Nx, Ny, Ns = imSize[0], imSize[2], imSize[1]
t1rMap = np.zeros((Ns, Nx, Ny), dtype='float32')
# t1rMap = np.zeros((Nx, Ns, Ny), dtype='float32')
ultraEfficient = 0

for ns in range(Ns):
    # work with smaller volumes for increased performance
    volBuffer = maskedTslBuffer[ns,:,:,:].copy('F') #take first set of TSLs played
#     volBuffer = maskedTslBuffer[:,ns,:,:].copy('F') #take first set of TSLs played
    maskedVolBuffer = np.squeeze(maskedTslBuffer[ns,:,:,0].copy('F'))
#     maskedVolBuffer1 = np.squeeze(maskedTslBuffer[ns,:,:,1].copy('F'))
#     maskedVolBuffer = np.squeeze(maskedTslBuffer[:,ns,:,0].copy('F'))
    t1rMapBuffer = np.array(np.zeros((Nx, Ny), dtype='float32', order='F'))
    
    nonZeroIndices = np.array(np.nonzero(maskedVolBuffer.ravel('F')))
#     nonZeroIndices1 = np.array(np.nonzero(maskedVolBuffer1.ravel('F')))
    
    dim_nonzero = len(nonZeroIndices.T)
#     dim_nonzero1 = len(nonZeroIndices1.T)
    
    if tslPlayed == 2:
        ultraEfficient = 1
    
    
    t1rMapBuffer = t1rhoCalc(dim_nonzero, nonZeroIndices, TSLs, volBuffer, t1rMapBuffer, ultraEfficient)
    
#     if dim_nonzero < dim_nonzero1:
#         t1rMapBuffer = t1rhoCalc(dim_nonzero, nonZeroIndices, TSLs, volBuffer, t1rMapBuffer, ultraEfficient)
#     else:
#         t1rMapBuffer = t1rhoCalc(dim_nonzero1, nonZeroIndices1, TSLs, volBuffer, t1rMapBuffer, ultraEfficient)

    t1rMap[ns,:,:] = t1rMapBuffer
#     t1rMap[:,ns,:] = t1rMapBuffer

t1rMap[t1rMap < 0] = 0
t1rMap[np.isinf(t1rMap)] = 0
t1rMap[t1rMap > np.median(t1rMap[t1rMap > 0])*10] = np.median(t1rMap[t1rMap > 0])
    

In [ ]:
print('t1rMask.shape: {0}\n t1rMap.shape: {1}'.format(t1rMask.shape, t1rMap.shape))
print('tslVol.shape: {0}'.format(tslVol.shape))

## Save T1rho map

In [ ]:
# subjDir = '20190807'
# subjDir = 'lysPhantom'
# parentDir = '/Users/nowusu/Desktop/FirstArticle_NKO/PHANTOM_20190807/SCANS/'
# outputName = parentDir + subjDir + '/' + subjDir + '_TSLs' + str(TSLs[0]) + str(TSLs[1]) + 'ms.nii'
# outputName = parentDir + subjDir + '/SCANS/fidall/test/t1rIm' + subjDir + '_TSLs' + str(TSLs[0]) + str(TSLs[1]) + 'ms.nii'

# t1rMask = nib.load("/Volumes/mrrcdata/Reliability_NKO/StudyData/20190321/SCANS/efgre/mask_T1EFGRE.filled.nii.gz").get_data()
# maskedT1r = applyMask(t1rMap,t1rMask)

# outputName = '/Users/nowusu/Documents/testRtest_scans/PHANTOM_20191206/SCANS/3dfse_fr1/3dfse_fr1.nii'
outputName = '/Users/nowusu/Documents/3T_t1rho/t1rho3DFSE_20200308/SCANS/non_selective/3dfse_non_selective.nii'

dcm2niConvert(imAffine, t1rMap, outputName)
# dcm2niConvert(imAffine, maskedT1r, outputName)

## Check images

In [ ]:
%matplotlib inline
from ipywidgets import interactive

def imSlider(whichSlice):
    plt.figure(figsize=(7,7))
    plt.imshow(t1rMap[whichSlice,:,:], vmin=10)
#     plt.imshow(np.rot90(maskedT1r[:,whichSlice,:]), vmin=10, vmax=300)
    plt.colorbar()
    plt.axis('off')
    plt.show()
    

interactive_plot = interactive(imSlider, whichSlice=(0, 92-1))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot


In [ ]:
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
# rc('text', usetex=True)
rc('font', weight='bold')
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']


fig, (tslIm1, tslIm2, t1rIm) = plt.subplots(1,3, figsize=(28,28))

im1 = tslIm1.imshow(np.rot90(maskedTslBuffer[:,101,:,0]), vmin=10)
im1.set_cmap('jet')
tslIm1.axis('off')
tslIm1.set_title('TSL 5ms', fontsize=30)

im2 = tslIm2.imshow(np.rot90(maskedTslBuffer[:,101,:,1]), vmin=10)
im2.set_cmap('jet')
tslIm2.axis('off')
tslIm2.set_title('TSL 80ms', fontsize=30)

im3 = t1rIm.imshow(np.rot90(maskedT1r[:,101,:]), vmin=10, vmax=200)
im3.set_cmap('jet')
t1rIm.axis('off')
t1rIm.set_title('T1rho', fontsize=30)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.812, 0.393, 0.025, 0.22])
fig.colorbar(im3, cax=cbar_ax)
# fig.axes[1].set_ylabel(r'\textbf{T1$\rho$ (ms)}', fontsize=25)
fig.axes[3].yaxis.set_tick_params(labelsize=20)

plt.subplots_adjust(wspace=0.05)
plt.show()

In [ ]:
rc('text', usetex=True)
rc('font', weight='bold')
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

fig2 = plt.figure(figsize=(7,7))
image = plt.subplot(1,1,1)
im1 = image.imshow(t1rMap[:,:,136], vmin=150, vmax=500)
im1.set_cmap('hot')
image.axis('off')
# image.set_title('T1rho Map')

fig2.subplots_adjust(right=0.8)
cbar_ax = fig2.add_axes([0.83, 0.15, 0.025, 0.70])
fig2.colorbar(im1, cax=cbar_ax)
fig2.axes[1].set_ylabel(r'\textbf{T1$\rho$ (ms)}', fontsize=20)
fig2.axes[1].yaxis.set_tick_params(labelsize=15)
plt.show()

In [ ]:
#fig.savefig('/Users/nowusu/Desktop/efgreISMRMFig.png', dpi=300, format='png')